In [9]:
%reload_ext dotenv
%dotenv

import os

config = {
    "server": os.environ["MSSQL_SERVER"],
    "database": os.environ["MSSQL_DATABASE"],
    "uid": os.environ["MSSQL_UID"],
    "pwd": os.environ["MSSQL_PWD"],
}

In [ ]:
import pyodbc

sql_server_drivers = list(filter(lambda x: "SQL Server" in x, pyodbc.drivers()))
conn_str = (
    f"DRIVER={{{sql_server_drivers[0]}}};"
    f'SERVER=tcp:{config["server"]};PORT=1433;'
    f'DATABASE={config["database"]};'
    f'UID={config["uid"]};'
    f'PWD={config["pwd"]}'
)
display(conn_str)

with pyodbc.connect(conn_str) as conn:
    with conn.cursor() as cursor:
        cursor.execute("select 1")

In [ ]:
import pytds

with pytds.connect(dsn=config["server"], database=config["database"], user=config["uid"], password=config["pwd"]) as conn:
    with conn.cursor() as cursor:
        cursor.execute("select 1")

In [ ]:
# import duckdb
import miniab as ab

source = ab.get_source(
    "source-mssql",
    config = config,
    sync = True
)
source.select_all_streams()
source.check()

cache = ab.get_default_cache()
result = source.read(cache=cache)

# dim_pack_tbl = result.to_arrow("dim_pack")
# duckdb.sql("SELECT * FROM dim_pack_tbl LIMIT 10;").show()